# 登录获取cookies
先要获取login页面上的lt值

In [3]:
import requests
from bs4 import BeautifulSoup

r = requests.get("http://cas.hdu.edu.cn/cas/login?service=http://jxgl.hdu.edu.cn/default.aspx")
soup = BeautifulSoup(r.text,"lxml")
lt = soup.findAll(class_="loginpage_center_row pwd")[0].findAll('input')[1]["value"]
print(lt)

LT-5959134-zEJhejAZvpq37XbVI5jA


## 向login页面发送post请求
请求表单包括用户名、密码等，这里的lt必须是login页面生成的值，否则会请求失败<br>
cookie为route、key_dcp_cas、dcp_cascookie<br>
新的cookies为CASTGC<br>
同时返回的页面上显示了一个带有ticket的url需要跳转

In [4]:
import requests

cookies_login={'route':'4376efc7edf61c9fe699e82a2fb7a34f','key_dcp_cas':'Y6xFbFMGyhQp29ZVVL3hf1dLP5HsSRyx84pzwythbt2Ftz6v8PTp!-1635894608','dcp_cascookie':'16196108||7ac025c952fdef2d24df3f40e0e8960e4de7951b2d79e96bf78e06ce8913ed6700baa3c3||true'}

headers = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'Accept-Encoding': 'gzip, deflate',
            'Accept-Language': 'zh-CN,zh;q=0.9',
           'Cache-Control': 'max-age=0',
            'Connection': 'keep-alive',
           'Content-Length': '259',
            'Content-Type': 'application/x-www-form-urlencoded',
           'Origin': 'http://cas.hdu.edu.cn',
           'Upgrade-Insecure-Requests': '1',
            'Host': 'cas.hdu.edu.cn',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
           'Referer':'http://cas.hdu.edu.cn/cas/login?service=http://jxgl.hdu.edu.cn/default.aspx'
          }

r = requests.post("http://cas.hdu.edu.cn/cas/login",
                 headers = headers,
                  cookies = cookies_login,
#                  data=params)
                 data='encodedService=http%253a%252f%252fjxgl.hdu.edu.cn%252fdefault.aspx&service=http%3A%2F%2Fjxgl.hdu.edu.cn%2Fdefault.aspx&serviceName=null&loginErrCnt=0&username=16196108&password=77023341bbe3d250643a708a971fac56&' \
                  +'lt=' + lt + '&autoLogin=true'
                 )

# r.encoding='gbk'
CASTGC = r.cookies.get_dict()
print(r.cookies.get_dict())
soup = BeautifulSoup(r.text,"lxml")
ticket_url = soup.find('a')['href']
print(soup.find('a')['href'])

{'CASTGC': 'TGC-5959135-psnTbb0oQh3dOzzAJ7Np99BwSODI3byfuJ60x3HKJTjDpXltni', 'key_dcp_cas': 'KBB9bGFXXBPxnth1LPPKbQGm7tQ3T1Ylc613Dtpt9JQNjX2CB1Px!-1635894608'}
http://jxgl.hdu.edu.cn/default.aspx?ticket=ST-5959136-9Zw1UldfjseBqo1mrgdT


## get上一个页面中要求跳转的url
cookies为上一个页面新的cookies：CASTGC<br>
新的cookies为route、key_dcp_cas<br>
同时返回的页面上显示了一个带有ticket的url需要跳转

In [5]:
r = requests.get(ticket_url,cookies=CASTGC)
print(r.cookies.get_dict())
soup = BeautifulSoup(r.text,"lxml")
ticket_url = soup.find('a')['href']
print(soup.find('a')['href'])

{'route': '4376efc7edf61c9fe699e82a2fb7a34f'}
http://jxdc.hdu.edu.cn/index.php?ticket=ST-5959139-VMqdIuT2gjTYu0hhk3SY


## 第二次get上一个页面中要求跳转的带有ticket的url
cookies为上一个页面新的cookies：route、key_dcp_cas<br>
新的cookies为PHPSESSID<br>
本次返回页面要求跳转到http://jxgl.hdu.edu.cn/index.aspx

In [6]:
cookies = r.cookies.get_dict()
r = requests.get(ticket_url,cookies=cookies)
print(r.cookies.get_dict())
print(r.text)
PHPSESSID = r.cookies.get_dict()

{'PHPSESSID': '078b30f10713b3ffd3edb56e3fadde84'}
<script language="javascript">location.href='http://jxgl.hdu.edu.cn/index.aspx';</script>


## get跳转到http://jxgl.hdu.edu.cn/index.aspx
本次cookies需要组装，分别是访问login是的三个cookies：key_dcp_cas、dcp_cascookie、route，和post请求返回的新cookies：CASTGC<br>
再次要求跳转到一个带有ticket的url...（第三个了）

In [69]:
# print(cookies_login)

In [7]:
# 将第一个login页面的cookies，和请求的cookies，重新组装
cookies_index = {'key_dcp_cas':cookies_login['key_dcp_cas'],'dcp_cascookie':cookies_login['dcp_cascookie'], \
                 'route':cookies_login['route'],'CASTGC': CASTGC['CASTGC']}

print(cookies_index)

{'route': '4376efc7edf61c9fe699e82a2fb7a34f', 'CASTGC': 'TGC-5959135-psnTbb0oQh3dOzzAJ7Np99BwSODI3byfuJ60x3HKJTjDpXltni', 'key_dcp_cas': 'Y6xFbFMGyhQp29ZVVL3hf1dLP5HsSRyx84pzwythbt2Ftz6v8PTp!-1635894608', 'dcp_cascookie': '16196108||7ac025c952fdef2d24df3f40e0e8960e4de7951b2d79e96bf78e06ce8913ed6700baa3c3||true'}


In [11]:
r = requests.get("http://jxgl.hdu.edu.cn/index.aspx",cookies=cookies_index)
# r = requests.get("http://jxgl.hdu.edu.cn/index.aspx",cookies=CASTGC)
print(r.cookies.get_dict())
# print(r.text)
soup = BeautifulSoup(r.text,"lxml")
ticket_url = soup.find('a')['href']
print(soup.find('a')['href'])

{'route': '4376efc7edf61c9fe699e82a2fb7a34f'}
http://jxgl.hdu.edu.cn/index.aspx?ticket=ST-5959163-z52J42QMJ3jvovCc882c


## 跳转到新的带有ticket的URL后，得到可以直接登录选课系统的cookies
ASP.NET_SessionId和route<br>
虽然返回的页面上还是要求跳转到一个url，但可以不管了<br>
执行这一段代码，会一直更新cookies

In [12]:
r = requests.get(ticket_url,cookies=PHPSESSID)
print(r.cookies.get_dict())
# print(r.text)

{'ASP.NET_SessionId': 'f1xj0m454fndqzv5ipeyhg45', 'route': '3d0c690852e4ff1f04aa9cef8f2994ef'}


## 利用cookies:ASP.NET_SessionId和route，直接get选课系统

In [8]:
cookies = r.cookies.get_dict()
r = requests.get("http://jxgl.hdu.edu.cn/xs_main.aspx?xh=16196108",cookies=cookies)
# print(r.text)

## 应用：爬成绩

In [9]:
headers = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'Accept-Encoding': 'gzip, deflate',
            'Accept-Language': 'zh-CN,zh;q=0.9',
           'Cache-Control': 'max-age=0',
            'Connection': 'keep-alive',
           'Content-Length': '7633',
            'Content-Type': 'application/x-www-form-urlencoded',
           'Origin': 'http://jxgl.hdu.edu.cn',
           'Upgrade-Insecure-Requests': '1',
            'Host': 'jxgl.hdu.edu.cn',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
           'Referer':'http://jxgl.hdu.edu.cn/xscjcx_dq.aspx?xh=16196108&xm=%u80e1%u9648%u6167&gnmkdm=N121605'# 修改
          }

params = {'__EVENTTARGET':'ddl_kcgs',
          'ddl_kcgs': '%CD%A8%CA%B6%B1%D8%D0%DE',
          'ddl_ywyl':'%D3%D0',
          '__VIEWSTATE':'%2FwEPDwUKLTcxMzExNjMwMw8WBh4FZHFzemoFBDIwMTYeBHp5ZG0FBDA1MjceCXNvcnRPcmRlcgUFIGFzYyAWAgIBD2QWFAIBDw8WAh4EVGV4dAVv5aeT5ZCN77ya6IOh6ZmI5oWnJm5ic3A7Jm5ic3A7Jm5ic3A7Jm5ic3A75a2m6Zmi77ya6K6h566X5py65a2m6ZmiJm5ic3A7Jm5ic3A7Jm5ic3A7Jm5ic3A75LiT5Lia77ya6L2v5Lu25bel56iLZGQCAw8QDxYGHg1EYXRhVGV4dEZpZWxkBQRrY3h6Hg5EYXRhVmFsdWVGaWVsZAUEa2N4eh4LXyFEYXRhQm91bmRnZBAVDQzliJvkuJrmlZnogrIM57uP566h5Lu76YCJG%2BenkeaKgOWPkeWxleS4juenkeWtpueyvuelngznp5HmioDku7vpgIkM5YW25a6D5Lu76YCJG%2BS6uuaWh%2Be7j%2BWFuOS4juS6uuaWh%2BS%2FruWFuwzkurrmlofku7vpgIkb56S%2B5Lya5Y%2BR5bGV5LiO5YWs5rCR5pWZ6IKyDOWkluivreaooeWdlxvmlofmmI7lr7nor53kuI7lm73pmYXop4bph44S5b%2BD55CG5YGl5bq36YCJ5L%2BuG%2BiJuuacr%2BWIm%2BS9nOS4juWuoee%2BjuS9k%2BmqjAAVDQzliJvkuJrmlZnogrIM57uP566h5Lu76YCJG%2BenkeaKgOWPkeWxleS4juenkeWtpueyvuelngznp5HmioDku7vpgIkM5YW25a6D5Lu76YCJG%2BS6uuaWh%2Be7j%2BWFuOS4juS6uuaWh%2BS%2FruWFuwzkurrmlofku7vpgIkb56S%2B5Lya5Y%2BR5bGV5LiO5YWs5rCR5pWZ6IKyDOWkluivreaooeWdlxvmlofmmI7lr7nor53kuI7lm73pmYXop4bph44S5b%2BD55CG5YGl5bq36YCJ5L%2BuG%2BiJuuacr%2BWIm%2BS9nOS4juWuoee%2BjuS9k%2BmqjAAUKwMNZ2dnZ2dnZ2dnZ2dnZ2RkAgUPEGQPFgNmAgECAhYDEAUD5pyJBQPmnIlnEAUD5pegBQPml6BnEGVlZxYBZmQCBw8QDxYGHwQFBGtjZ3MfBQUEa2Nncx8GZ2QQFQsM5Yib5paw5a6e6Le1DOWIm%2BaWsOmAieS%2Frg%2FliJvkuJrpgJror4bor74P5Yib5Lia6YCJ5L%2Bu6K%2B%2BDOmAmuivhuW%2FheS%2FrhXpgJror4bpgInkv67moLjlv4Por74V6YCa6K%2BG6YCJ5L%2Bu5LiA6Iis6K%2B%2BD%2BagoemZhemAieS%2FruivvhXlv4PnkIblgaXlurfpgInkv67or74S5pm65oWn5qCR5re35ZCI5byPABULDOWIm%2BaWsOWunui3tQzliJvmlrDpgInkv64P5Yib5Lia6YCa6K%2BG6K%2B%2BD%2BWIm%2BS4mumAieS%2FruivvgzpgJror4blv4Xkv64V6YCa6K%2BG6YCJ5L%2Bu5qC45b%2BD6K%2B%2BFemAmuivhumAieS%2FruS4gOiIrOivvg%2FmoKHpmYXpgInkv67or74V5b%2BD55CG5YGl5bq36YCJ5L%2Bu6K%2B%2BEuaZuuaFp%2Bagkea3t%2BWQiOW8jwAUKwMLZ2dnZ2dnZ2dnZ2cWAQIBZAIJDxAPFgYfBAUEeHFtYx8FBQR4cWRtHwZnZBAVBgbkuIvmspkG5Lic5bKzBuaWh%2BS4gAbkuI3lrpoJ5Lic5qCh5Yy6CemdkuWxsea5lhUGATEBMgEzATQBNQE2FCsDBmdnZ2dnZxYBZmQCCw8QDxYGHwQFBHNrc2ofBQUEc2tzah8GZ2QQFQUk5ZGo5Zub56ysMTAsMTHoioJ756ysMS0xN%2BWRqHzlj4zlkah9JOWRqOS4gOesrDEwLDEx6IqCe%2BesrDEtMTflkah85Y%2BM5ZGofSLlkajkuIDnrKw4LDnoioJ756ysMS0xN%2BWRqHzlj4zlkah9G%2BWRqOS4gOesrDgsOeiKgnvnrKw1LTEy5ZGofQAVBSTlkajlm5vnrKwxMCwxMeiKgnvnrKwxLTE35ZGofOWPjOWRqH0k5ZGo5LiA56ysMTAsMTHoioJ756ysMS0xN%2BWRqHzlj4zlkah9IuWRqOS4gOesrDgsOeiKgnvnrKwxLTE35ZGofOWPjOWRqH0b5ZGo5LiA56ysOCw56IqCe%2BesrDUtMTLlkah9ABQrAwVnZ2dnZxYBAgRkAhEPDxYCHgdWaXNpYmxlaGRkAhYPPCsACwEADxYIHghEYXRhS2V5cxYAHgtfIUl0ZW1Db3VudGYeCVBhZ2VDb3VudAIBHhVfIURhdGFTb3VyY2VJdGVtQ291bnRmZGQCGA88KwALAQAPFggfCBYAHwkCAh8KAgEfCwICZBYCZg9kFgQCAQ9kFiBmDw8WAh8DBR4oMjAxOC0yMDE5LTEpLUMwMzk5MDA2LTQxNTc5LTFkZAIBDw8WAh8DBQzliJvkuJrnrqHnkIZkZAICDw8WAh8DBQnlvKDntKDlubNkZAIDDw8WAh8DBQMyLjBkZAIEDw8WAh8DBQcyLjAtMC4wZGQCBQ8PFgIfAwUFMDEtMTdkZAIGDw8WAh8DBQbkuIvmsplkZAIHDw8WAh8DBRvlkajkuoznrKw4LDnoioJ756ysMS0xN%2BWRqH1kZAIIDw8WAh8DBRPnrKw35pWZ56CU5qW85YyXMTE2ZGQCCQ8PFgIfAwUGJm5ic3A7ZGQCCg8PFgIfAwUBMGRkAgsPDxYCHwMFD%2BWIm%2BS4mumAmuivhuivvmRkAgwPDxYCHwMFDOWIm%2BS4muaVmeiCsmRkAg0PDxYCHwMFBuS4i%2BaymWRkAg4PDxYCHwMFBiZuYnNwO2RkAg8PDxYCHwMFBiZuYnNwO2RkAgIPZBYgZg8PFgIfAwUeKDIwMTgtMjAxOS0xKS1DNTY5MjAxNC0wNDEwMC0xZGQCAQ8PFgIfAwUY5LiW55WM5rCR5peP6Z%2Bz5LmQ5paH5YyWZGQCAg8PFgIfAwUJ5ZCV5Lic5pa5ZGQCAw8PFgIfAwUDMi4wZGQCBA8PFgIfAwUHMi4wLTAuMGRkAgUPDxYCHwMFBTAxLTE3ZGQCBg8PFgIfAwUG5LiL5rKZZGQCBw8PFgIfAwUb5ZGo5LiA56ysOCw56IqCe%2BesrDEtMTflkah9ZGQCCA8PFgIfAwUQ56ysM%2BaVmeeglOalvDEwNWRkAgkPDxYCHwMFBiZuYnNwO2RkAgoPDxYCHwMFATFkZAILDw8WAh8DBRXpgJror4bpgInkv67kuIDoiKzor75kZAIMDw8WAh8DBRvoibrmnK%2FliJvkvZzkuI7lrqHnvo7kvZPpqoxkZAINDw8WAh8DBQbkuIvmsplkZAIODw8WAh8DBQYmbmJzcDtkZAIPDw8WAh8DBQYmbmJzcDtkZAIdDzwrAAsBAA8WAh8HaGRkZA%3D%3D',
          '__EVENTVALIDATION':'%2FwEWNQK%2BjfC0AgKP3%2B6lAgL91OO8DAK7isW0CgK%2Bt%2Fm1AQLNiMH%2FCgKS18q3CALJ27XlCgLm6%2FqHCgKhjbWVAQKA4u2DAwK2ga2CCgLQm9zvAwKCltOYCgKP3%2B6lAgKYyPTRDwK%2F4d%2B%2FAwLY7N%2B%2FAwKYyPTRDwKQrrE3AqmAsvEKAvLQ1bYHAsaAvr0HAvv236sBAveGxuwEAsuOjKoGAvPy%2BY8JAsqUzLUEAqrupp0JAt3ihPIMApCusTcC0OnS9g4C34b4mAIC3ob4mAIC3Yb4mAIC3Ib4mAIC24b4mAIC2ob4mAICxL%2Ffhw4CoY6Mgg4C66L56gEC6O2c7goCvoP7ww0CxL%2Ffhw4C7NGy6wYCu6uxhggChdn12wMCoMKT8Q0C2YiFpw8CvIH66gICkvvWnAsCjOeKxgYC05Tgxg0%3D',
          'ddl_xqbs':'1',
          'hidXNXQ':'2018-20191'
    }
xn = '2017-2018'
xq = '1'

xn = input('请输入学年：')
xq = input('请输入学期：')
r = requests.post("http://jxgl.hdu.edu.cn/xscjcx_dq.aspx?xh=16196108&xm=%u80e1%u9648%u6167&gnmkdm=N121605",# 修改
                 cookies=cookies,
                 headers=headers,
                 data='__EVENTTARGET=&__EVENTARGUMENT=&__LASTFOCUS=&__VIEWSTATE=%2FwEPDwULLTIxMDUwNTQwMjIPZBYCAgEPZBYGAgEPEGQQFRMACTIwMDEtMjAwMgkyMDAyLTIwMDMJMjAwMy0yMDA0CTIwMDQtMjAwNQkyMDA1LTIwMDYJMjAwNi0yMDA3CTIwMDctMjAwOAkyMDA4LTIwMDkJMjAwOS0yMDEwCTIwMTAtMjAxMQkyMDExLTIwMTIJMjAxMi0yMDEzCTIwMTMtMjAxNAkyMDE0LTIwMTUJMjAxNS0yMDE2CTIwMTYtMjAxNwkyMDE3LTIwMTgJMjAxOC0yMDE5FRMACTIwMDEtMjAwMgkyMDAyLTIwMDMJMjAwMy0yMDA0CTIwMDQtMjAwNQkyMDA1LTIwMDYJMjAwNi0yMDA3CTIwMDctMjAwOAkyMDA4LTIwMDkJMjAwOS0yMDEwCTIwMTAtMjAxMQkyMDExLTIwMTIJMjAxMi0yMDEzCTIwMTMtMjAxNAkyMDE0LTIwMTUJMjAxNS0yMDE2CTIwMTYtMjAxNwkyMDE3LTIwMTgJMjAxOC0yMDE5FCsDE2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dkZAIHD2QWBmYPZBYCZg8WAh4JaW5uZXJodG1sBSUyMDE3LTIwMTjlrablubTnrKwx5a2m5pyf5a2m5Lmg5oiQ57upZAIBD2QWBmYPFgIfAAUR5a2m5Y%2B377yaMTYxOTYxMDhkAgEPFgIfAAUS5aeT5ZCN77ya6IOh6ZmI5oWnZAICDxYCHwAFGOWtpumZou%2B8muiuoeeul%2BacuuWtpumZomQCAg9kFgRmDxYCHwAFFeS4k%2BS4mu%2B8mui9r%2BS7tuW3peeoi2QCAQ8WAh8ABRTooYzmlL%2Fnj63vvJoxNjA1MjcxMWQCCQ88KwALAQAPFggeCERhdGFLZXlzFgAeC18hSXRlbUNvdW50AhEeCVBhZ2VDb3VudAIBHhVfIURhdGFTb3VyY2VJdGVtQ291bnQCEWQWAmYPZBYiAgEPZBYcZg8PFgIeBFRleHQFHigyMDE3LTIwMTgtMSktQTExMDIwODAtNDA0NDMtNWRkAgEPDxYCHwUFCTIwMTctMjAxOGRkAgIPDxYCHwUFATFkZAIDDw8WAh8FBQhBMTEwMjA4MGRkAgQPDxYCHwUFD%2Bi3qOaWh%2BWMluS6pOmZhWRkAgUPDxYCHwUFDOWkluivreaooeWdl2RkAgYPDxYCHwUFDOWFrOWFseW%2FheS%2FrmRkAgcPDxYCHwUFAzIuMGRkAggPDxYCHwUFAjg3ZGQCCQ8PFgIfBQUGJm5ic3A7ZGQCCg8PFgIfBQUGJm5ic3A7ZGQCCw8PFgIfBQUP5aSW5Zu96K%2Bt5a2m6ZmiZGQCDA8PFgIfBQUGJm5ic3A7ZGQCDQ8PFgIfBQUGJm5ic3A7ZGQCAg9kFhxmDw8WAh8FBR4oMjAxNy0yMDE4LTEpLUMwMzk5MDI0LTQxNjY5LTFkZAIBDw8WAh8FBQkyMDE3LTIwMThkZAICDw8WAh8FBQExZGQCAw8PFgIfBQUIQzAzOTkwMjRkZAIEDw8WAh8FBRXlpKflrabnlJ%2FliJvkuJrln7rnoYBkZAIFDw8WAh8FBQzliJvkuJrmlZnogrJkZAIGDw8WAh8FBQYmbmJzcDtkZAIHDw8WAh8FBQMxLjBkZAIIDw8WAh8FBQI5OWRkAgkPDxYCHwUFBiZuYnNwO2RkAgoPDxYCHwUFBiZuYnNwO2RkAgsPDxYCHwUFBiZuYnNwO2RkAgwPDxYCHwUFBiZuYnNwO2RkAg0PDxYCHwUFBiZuYnNwO2RkAgMPZBYcZg8PFgIfBQUeKDIwMTctMjAxOC0xKS1BMDcxNDA0MC00MDEyNS0xZGQCAQ8PFgIfBQUJMjAxNy0yMDE4ZGQCAg8PFgIfBQUBMWRkAgMPDxYCHwUFCEEwNzE0MDQwZGQCBA8PFgIfBQUY5qaC546H6K665LiO5pWw55CG57uf6K6hZGQCBQ8PFgIfBQUM5YWs5YWx5b%2BF5L%2BuZGQCBg8PFgIfBQUGJm5ic3A7ZGQCBw8PFgIfBQUDMy4wZGQCCA8PFgIfBQUCODdkZAIJDw8WAh8FBQYmbmJzcDtkZAIKDw8WAh8FBQYmbmJzcDtkZAILDw8WAh8FBQnnkIblrabpmaJkZAIMDw8WAh8FBQYmbmJzcDtkZAINDw8WAh8FBQYmbmJzcDtkZAIED2QWHGYPDxYCHwUFHigyMDE3LTIwMTgtMSktQTIzMDEwMjAtNDA3MTItMmRkAgEPDxYCHwUFCTIwMTctMjAxOGRkAgIPDxYCHwUFATFkZAIDDw8WAh8FBQhBMjMwMTAyMGRkAgQPDxYCHwUFG%2BmprOWFi%2BaAneS4u%2BS5ieWfuuacrOWOn%2BeQhmRkAgUPDxYCHwUFDOWFrOWFseW%2FheS%2FrmRkAgYPDxYCHwUFBiZuYnNwO2RkAgcPDxYCHwUFAzIuMGRkAggPDxYCHwUFAjkxZGQCCQ8PFgIfBQUGJm5ic3A7ZGQCCg8PFgIfBQUGJm5ic3A7ZGQCCw8PFgIfBQUV6ams5YWL5oCd5Li75LmJ5a2m6ZmiZGQCDA8PFgIfBQUGJm5ic3A7ZGQCDQ8PFgIfBQUGJm5ic3A7ZGQCBQ9kFhxmDw8WAh8FBR4oMjAxNy0yMDE4LTEpLUEyMzAxMDExLWxzbTk0LTFkZAIBDw8WAh8FBQkyMDE3LTIwMThkZAICDw8WAh8FBQExZGQCAw8PFgIfBQUIQTIzMDEwMTFkZAIEDw8WAh8FBT3mr5vms73kuJzmgJ3mg7PlkozkuK3lm73nibnoibLnpL7kvJrkuLvkuYnnkIborrrkvZPns7vmpoLorroxZGQCBQ8PFgIfBQUM5YWs5YWx5b%2BF5L%2BuZGQCBg8PFgIfBQUGJm5ic3A7ZGQCBw8PFgIfBQUDMy4wZGQCCA8PFgIfBQUCODRkZAIJDw8WAh8FBQYmbmJzcDtkZAIKDw8WAh8FBQYmbmJzcDtkZAILDw8WAh8FBRXpqazlhYvmgJ3kuLvkuYnlrabpmaJkZAIMDw8WAh8FBQYmbmJzcDtkZAINDw8WAh8FBQYmbmJzcDtkZAIGD2QWHGYPDxYCHwUFHigyMDE3LTIwMTgtMSktVzAwMDExMzEtNDEzNzAtMWRkAgEPDxYCHwUFCTIwMTctMjAxOGRkAgIPDxYCHwUFATFkZAIDDw8WAh8FBQhXMDAwMTEzMWRkAgQPDxYCHwUFJeWkp%2BWtpueUn%2BiBjOS4muWPkeWxleS4juWwseS4muaMh%2BWvvDFkZAIFDw8WAh8FBQzor77lpJblv4Xkv65kZAIGDw8WAh8FBQYmbmJzcDtkZAIHDw8WAh8FBQMxLjBkZAIIDw8WAh8FBQI5NWRkAgkPDxYCHwUFBiZuYnNwO2RkAgoPDxYCHwUFBiZuYnNwO2RkAgsPDxYCHwUFNOacrOenkeeUn%2BmZouOAgeWFmuWnlOWtpueUn%2BW3peS9nOmDqC3mi5vnlJ%2FlsLHkuJrlpIRkZAIMDw8WAh8FBQYmbmJzcDtkZAINDw8WAh8FBQYmbmJzcDtkZAIHD2QWHGYPDxYCHwUFHigyMDE3LTIwMTgtMSktVzAwMDEwMzEtNDA1MDktMWRkAgEPDxYCHwUFCTIwMTctMjAxOGRkAgIPDxYCHwUFATFkZAIDDw8WAh8FBQhXMDAwMTAzMWRkAgQPDxYCHwUFEOW9ouWKv%2BS4juaUv%2BetljFkZAIFDw8WAh8FBQzor77lpJblv4Xkv65kZAIGDw8WAh8FBQYmbmJzcDtkZAIHDw8WAh8FBQMwLjVkZAIIDw8WAh8FBQbkvJjnp4BkZAIJDw8WAh8FBQYmbmJzcDtkZAIKDw8WAh8FBQYmbmJzcDtkZAILDw8WAh8FBRXpqazlhYvmgJ3kuLvkuYnlrabpmaJkZAIMDw8WAh8FBQYmbmJzcDtkZAINDw8WAh8FBQYmbmJzcDtkZAIID2QWHGYPDxYCHwUFHigyMDE3LTIwMTgtMSktUzA1MDEyNjAtNDE2MzItMWRkAgEPDxYCHwUFCTIwMTctMjAxOGRkAgIPDxYCHwUFATFkZAIDDw8WAh8FBQhTMDUwMTI2MGRkAgQPDxYCHwUFGOeoi%2BW6j%2Biuvuiuoeivvueoi%2BiuvuiuoWRkAgUPDxYCHwUFBuWunui3tWRkAgYPDxYCHwUFBiZuYnNwO2RkAgcPDxYCHwUFAzEuMGRkAggPDxYCHwUFBuS8mOengGRkAgkPDxYCHwUFBiZuYnNwO2RkAgoPDxYCHwUFBiZuYnNwO2RkAgsPDxYCHwUFD%2Biuoeeul%2BacuuWtpumZomRkAgwPDxYCHwUFBiZuYnNwO2RkAg0PDxYCHwUFBiZuYnNwO2RkAgkPZBYcZg8PFgIfBQUeKDIwMTctMjAxOC0xKS1TMDUwMjI1MC00MTIwMi0xZGQCAQ8PFgIfBQUJMjAxNy0yMDE4ZGQCAg8PFgIfBQUBMWRkAgMPDxYCHwUFCFMwNTAyMjUwZGQCBA8PFgIfBQUY5pWw5o2u57uT5p6E6K%2B%2B56iL6K6%2B6K6hZGQCBQ8PFgIfBQUG5a6e6Le1ZGQCBg8PFgIfBQUGJm5ic3A7ZGQCBw8PFgIfBQUDMS4wZGQCCA8PFgIfBQUCOTVkZAIJDw8WAh8FBQYmbmJzcDtkZAIKDw8WAh8FBQYmbmJzcDtkZAILDw8WAh8FBQ%2ForqHnrpfmnLrlrabpmaJkZAIMDw8WAh8FBQYmbmJzcDtkZAINDw8WAh8FBQYmbmJzcDtkZAIKD2QWHGYPDxYCHwUFHigyMDE3LTIwMTgtMSktUzA1MDc5MDAtNDAzOTItMWRkAgEPDxYCHwUFCTIwMTctMjAxOGRkAgIPDxYCHwUFATFkZAIDDw8WAh8FBQhTMDUwNzkwMGRkAgQPDxYCHwUFFeaVsOaNruW6k%2Bivvueoi%2BiuvuiuoWRkAgUPDxYCHwUFBuWunui3tWRkAgYPDxYCHwUFBiZuYnNwO2RkAgcPDxYCHwUFAzEuMGRkAggPDxYCHwUFBuiJr%2BWlvWRkAgkPDxYCHwUFBiZuYnNwO2RkAgoPDxYCHwUFBiZuYnNwO2RkAgsPDxYCHwUFD%2Biuoeeul%2BacuuWtpumZomRkAgwPDxYCHwUFBiZuYnNwO2RkAg0PDxYCHwUFBiZuYnNwO2RkAgsPZBYcZg8PFgIfBQUeKDIwMTctMjAxOC0xKS1UMTMwMDAwMS0wNDE1Ni0zZGQCAQ8PFgIfBQUJMjAxNy0yMDE4ZGQCAg8PFgIfBQUBMWRkAgMPDxYCHwUFCFQxMzAwMDAxZGQCBA8PFgIfBQUS5L2T6IKyLeevrueQgyjnlLcpZGQCBQ8PFgIfBQUM5qCh5a6a5b%2BF5L%2BuZGQCBg8PFgIfBQUGJm5ic3A7ZGQCBw8PFgIfBQUDMS4wZGQCCA8PFgIfBQUCOTNkZAIJDw8WAh8FBQYmbmJzcDtkZAIKDw8WAh8FBQYmbmJzcDtkZAILDw8WAh8FBQ%2FkvZPogrLmlZnlrabpg6hkZAIMDw8WAh8FBQYmbmJzcDtkZAINDw8WAh8FBQYmbmJzcDtkZAIMD2QWHGYPDxYCHwUFHigyMDE3LTIwMTgtMSktQTA1MDcwNDItNDAzMjEtMWRkAgEPDxYCHwUFCTIwMTctMjAxOGRkAgIPDxYCHwUFATFkZAIDDw8WAh8FBQhBMDUwNzA0MmRkAgQPDxYCHwUFDeemu%2BaVo%2BaVsOWtpjJkZAIFDw8WAh8FBQzlrabnp5Hlv4Xkv65kZAIGDw8WAh8FBQYmbmJzcDtkZAIHDw8WAh8FBQMyLjBkZAIIDw8WAh8FBQI4MmRkAgkPDxYCHwUFBiZuYnNwO2RkAgoPDxYCHwUFBiZuYnNwO2RkAgsPDxYCHwUFD%2Biuoeeul%2BacuuWtpumZomRkAgwPDxYCHwUFBiZuYnNwO2RkAg0PDxYCHwUFBiZuYnNwO2RkAg0PZBYcZg8PFgIfBQUeKDIwMTctMjAxOC0xKS1BMDUwNzAyMC00MTIwMi0xZGQCAQ8PFgIfBQUJMjAxNy0yMDE4ZGQCAg8PFgIfBQUBMWRkAgMPDxYCHwUFCEEwNTA3MDIwZGQCBA8PFgIfBQUV5pWw5o2u57uT5p6E77yI55Sy77yJZGQCBQ8PFgIfBQUM5a2m56eR5b%2BF5L%2BuZGQCBg8PFgIfBQUGJm5ic3A7ZGQCBw8PFgIfBQUDNC4wZGQCCA8PFgIfBQUCOTJkZAIJDw8WAh8FBQYmbmJzcDtkZAIKDw8WAh8FBQYmbmJzcDtkZAILDw8WAh8FBQ%2ForqHnrpfmnLrlrabpmaJkZAIMDw8WAh8FBQYmbmJzcDtkZAINDw8WAh8FBQYmbmJzcDtkZAIOD2QWHGYPDxYCHwUFHigyMDE3LTIwMTgtMSktQTA3MTUwNTItNDAxMzItMmRkAgEPDxYCHwUFCTIwMTctMjAxOGRkAgIPDxYCHwUFATFkZAIDDw8WAh8FBQhBMDcxNTA1MmRkAgQPDxYCHwUFH%2BeJqeeQhuWtpuWOn%2BeQhuWPiuW3peeoi%2BW6lOeUqDJkZAIFDw8WAh8FBQzlrabnp5Hlv4Xkv65kZAIGDw8WAh8FBQYmbmJzcDtkZAIHDw8WAh8FBQMzLjBkZAIIDw8WAh8FBQI5MWRkAgkPDxYCHwUFBiZuYnNwO2RkAgoPDxYCHwUFBiZuYnNwO2RkAgsPDxYCHwUFCeeQhuWtpumZomRkAgwPDxYCHwUFBiZuYnNwO2RkAg0PDxYCHwUFBiZuYnNwO2RkAg8PZBYcZg8PFgIfBQUeKDIwMTctMjAxOC0xKS1BMDUwODIwMC00MDM5MC0xZGQCAQ8PFgIfBQUJMjAxNy0yMDE4ZGQCAg8PFgIfBQUBMWRkAgMPDxYCHwUFCEEwNTA4MjAwZGQCBA8PFgIfBQUbV0VC5bqU55So56iL5bqP6K6%2B6K6hKEpBVkEpZGQCBQ8PFgIfBQUM5LiT5Lia5b%2BF5L%2BuZGQCBg8PFgIfBQUGJm5ic3A7ZGQCBw8PFgIfBQUDNC4wZGQCCA8PFgIfBQUCOTNkZAIJDw8WAh8FBQYmbmJzcDtkZAIKDw8WAh8FBQYmbmJzcDtkZAILDw8WAh8FBQ%2ForqHnrpfmnLrlrabpmaJkZAIMDw8WAh8FBQYmbmJzcDtkZAINDw8WAh8FBQYmbmJzcDtkZAIQD2QWHGYPDxYCHwUFHigyMDE3LTIwMTgtMSktQTA1MDI1NzAtNDAzOTItMWRkAgEPDxYCHwUFCTIwMTctMjAxOGRkAgIPDxYCHwUFATFkZAIDDw8WAh8FBQhBMDUwMjU3MGRkAgQPDxYCHwUFCeaVsOaNruW6k2RkAgUPDxYCHwUFDOS4k%2BS4muW%2FheS%2FrmRkAgYPDxYCHwUFBiZuYnNwO2RkAgcPDxYCHwUFAzMuMGRkAggPDxYCHwUFAjkwZGQCCQ8PFgIfBQUGJm5ic3A7ZGQCCg8PFgIfBQUGJm5ic3A7ZGQCCw8PFgIfBQUP6K6h566X5py65a2m6ZmiZGQCDA8PFgIfBQUGJm5ic3A7ZGQCDQ8PFgIfBQUGJm5ic3A7ZGQCEQ9kFhxmDw8WAh8FBR4oMjAxNy0yMDE4LTEpLUIwNTAyNzQwLTA2MDQyLTFkZAIBDw8WAh8FBQkyMDE3LTIwMThkZAICDw8WAh8FBQExZGQCAw8PFgIfBQUIQjA1MDI3NDBkZAIEDw8WAh8FBQ9DKyvnqIvluo%2Forr7orqFkZAIFDw8WAh8FBQzkuJPkuJrpmZDpgIlkZAIGDw8WAh8FBQYmbmJzcDtkZAIHDw8WAh8FBQM0LjBkZAIIDw8WAh8FBQI5NWRkAgkPDxYCHwUFBiZuYnNwO2RkAgoPDxYCHwUFBiZuYnNwO2RkAgsPDxYCHwUFD%2Biuoeeul%2BacuuWtpumZomRkAgwPDxYCHwUFBiZuYnNwO2RkAg0PDxYCHwUFBiZuYnNwO2RkZA%3D%3D&__EVENTVALIDATION=%2FwEWGgLztZ%2FWBwKOwemfDgKOwemfDgKc6PHxDgKf6O1nApbomfIPApnotegBApjoofIMApvo3egOApLoyfINApXopYsNAprozbADAsCqyt4FAsOqjp8DAsKqkt8CAt2q1h8C3Kq63wMC36r%2BnwEC3qrCXwLZqobgAQLYqqrCDgL%2FwOmfDgL%2FwOmfDgLwr8PxAgLxr8PxAgLwksmiDg%3D%3D' \
                  +'&ddlxn=' + xn \
                  +'&ddlxq=' + xq \
                  +'&btnCx=+%B2%E9++%D1%AF+'
                 )

# r.encoding='gbk'
exam = BeautifulSoup(r.text,"lxml")

# print(soup.get_text())
t = exam.find_all(class_="datelist")[0].get_text()

# print(t)
#定义类，循环存储 
class lesson():
    def __init__(self,name,id,type_,credit,score,college):
        self.name = name
        self.id = id
        self.type = type
        self.credit = credit
        self.score = score
        self.college = college
    def show(self):
        print(self.id,'  ',end=' ')
#          print(self.id,'\t',self.name,'\t\t',self.type,'\t',self.credit,'\t',self.score,'\t',self.college,'\t')
#         print('%-10s%-20s%-10s%-10s%-10s%-10s'%(self.id,self.name,self.type,self.credit,self.score,self.college))
        print('{name:<{len}}'.format(name=self.name,len=45-len(self.name.encode('GBK'))+len(self.name)),end=' ')
        print('{type:<{len}}'.format(type=self.type,len=10-len(self.type.encode('GBK'))+len(self.type)),end=' ')
        print(self.credit,'\t',self.score,'\t',self.college,'\t')
exam_data = []

lesson_data_list = exam.find_all(class_="datelist")[0].find_all("tr")[1:] #[0]是标题 从[1]开始是课程信息
for lesson_data in lesson_data_list:
    td = lesson_data.find_all("td")
    name = td[3].get_text()
    id = td[2].get_text()
    type = td[4].get_text()
    credit = td[6].get_text()
    score = td[7].get_text()
    college = td[10].get_text()
    exam_data.append(lesson(name,id,type,credit,score,college))

#格式化，循环输出
print('\t\t\t\t\t%s学年第%s学期学习成绩'%(xn,xq))
print('课程代码   ',end=' ')
print('{name:<{len}}'.format(name='课程名称',len=45-len('课程名称'.encode('GBK'))+len('课程名称')),end=' ')
print('{type:<{len}}'.format(type='课程性质',len=10-len('课程性质'.encode('GBK'))+len('课程性质')),end=' ')
print('学分\t 成绩\t 开课学院\t')

for lesson_data in exam_data:
    lesson_data.show()

请输入学年：2017-2018
请输入学期：1
					2017-2018学年第1学期学习成绩
课程代码    课程名称                                      课程性质   学分	 成绩	 开课学院	
A1102080    跨文化交际                                    外语模块   2.0 	 87 	 外国语学院 	
C0399024    大学生创业基础                                创业教育   1.0 	 99 	   	
A0714040    概率论与数理统计                              公共必修   3.0 	 87 	 理学院 	
A2301020    马克思主义基本原理                            公共必修   2.0 	 91 	 马克思主义学院 	
A2301011    毛泽东思想和中国特色社会主义理论体系概论1     公共必修   3.0 	 84 	 马克思主义学院 	
W0001131    大学生职业发展与就业指导1                     课外必修   1.0 	 95 	 本科生院、党委学生工作部-招生就业处 	
W0001031    形势与政策1                                   课外必修   0.5 	 优秀 	 马克思主义学院 	
S0501260    程序设计课程设计                              实践       1.0 	 优秀 	 计算机学院 	
S0502250    数据结构课程设计                              实践       1.0 	 95 	 计算机学院 	
S0507900    数据库课程设计                                实践       1.0 	 良好 	 计算机学院 	
T1300001    体育-篮球(男)                                 校定必修   1.0 	 93 	 体育教学部 	
A0507042    离散数学2              

In [11]:
import requests
params = {'username': 'Ryan', 'password': 'password'}
r = requests.post("http://pythonscraping.com/pages/cookies/welcome.php", params)
print("Cookie is set to:")
print(r.cookies.get_dict())
print("-----------")
print("Going to profile page...")

Cookie is set to:
{'username': 'Ryan', 'loggedin': '1'}
-----------
Going to profile page...
